In [ ]:
!pip install factor_analyzer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Tratamiento de datos(3 paqueterías: numpy, pandas, statsmodels.api)
# ==============================================================================
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Preprocesado y modelado (3 paqueterías sklearn.preprocessing para StandardScaler, scale y factor_analyzer para el análisis factorial)
# ==============================================================================
from factor_analyzer import FactorAnalyzer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [ ]:
iris = sm.datasets.get_rdataset("iris", "datasets")
datos = iris.data

In [ ]:
datos.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [ ]:
#es muy importante que nuestros datos tengan media 0
print("Media de cada variable")
datos.mean()

Media de cada variable


Sepal.Length    5.843333
Sepal.Width     3.057333
Petal.Length    3.758000
Petal.Width     1.199333
dtype: float64

In [ ]:
#desviación estándar debe ser de 1
print("Desviación estándar de cada variable")
datos.std()

Desviación estándar de cada variable


Sepal.Length    0.828066
Sepal.Width     0.435866
Petal.Length    1.765298
Petal.Width     0.762238
dtype: float64

In [ ]:
escalar = StandardScaler()

In [ ]:
#Separar x de Y
features = ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width']
# Separating out the features
x = datos.loc[:, features]
# Separating out the target 
y = datos.loc[:,['Species']]

In [ ]:
datos_escalados = escalar.fit(x)
datos_escalados = escalar.transform(x)
datos_escalados

array([[-9.00681170e-01,  1.01900435e+00, -1.34022653e+00,
        -1.31544430e+00],
       [-1.14301691e+00, -1.31979479e-01, -1.34022653e+00,
        -1.31544430e+00],
       [-1.38535265e+00,  3.28414053e-01, -1.39706395e+00,
        -1.31544430e+00],
       [-1.50652052e+00,  9.82172869e-02, -1.28338910e+00,
        -1.31544430e+00],
       [-1.02184904e+00,  1.24920112e+00, -1.34022653e+00,
        -1.31544430e+00],
       [-5.37177559e-01,  1.93979142e+00, -1.16971425e+00,
        -1.05217993e+00],
       [-1.50652052e+00,  7.88807586e-01, -1.34022653e+00,
        -1.18381211e+00],
       [-1.02184904e+00,  7.88807586e-01, -1.28338910e+00,
        -1.31544430e+00],
       [-1.74885626e+00, -3.62176246e-01, -1.34022653e+00,
        -1.31544430e+00],
       [-1.14301691e+00,  9.82172869e-02, -1.28338910e+00,
        -1.44707648e+00],
       [-5.37177559e-01,  1.47939788e+00, -1.28338910e+00,
        -1.31544430e+00],
       [-1.26418478e+00,  7.88807586e-01, -1.22655167e+00,
      

In [ ]:
#Prueba de Barlett para saber si utilizar FA es buena opción

from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
chi_square_value, p_value=calculate_bartlett_sphericity(datos_escalados)
chi_square_value, p_value

#se puede observar un valor de p menor que 0.05, por lo tanto, 
#nuestra matriz de correlación es significativa y quiere decir que es muy probable que haya una relación subyacente entre algunas de mis variables.  


(706.9592430234728, 1.9226796044169772e-149)

In [ ]:
#es necesario determinar cuántos factores construir, por lo tanto,  
#se corre el siguiente código:
#Crear un análisis factorial sin especificaciones de factores para poder extraer
#los valores de los eigenvalores y determinar el número de factores a utilizar
fa = FactorAnalyzer()
#crear un análisis facotrial sin especificaciones de número de factores 
fa.fit(datos_escalados)
#Check eigenvalues
ev, v = fa.get_eigenvalues()
ev

array([2.91849782, 0.91403047, 0.14675688, 0.02071484])

De esta manera, podemos observar que existen 2 escenarios que podríamos considerar,  
el primero sería construir únicamente un factor dado que únicamente existe un 
eigenvalor con valor superior a 1 y, el segundo sería construir 2 factores dado que los 
valores de sus respectivos eigenvalores son muy cercanos a 1. Por lo tanto, vamos a 
correr ambos escenarios y analizar la varianza que se explica en cada uno para poder 
tomar una decisión.   
Como se mencionó el siguiente paso es hacer la construcción de los 2 escenarios, por lo 
tanto, se corren los siguientes códigos:  

In [ ]:
#Primer escenario
fa = FactorAnalyzer(n_factors=1) # 1 o 2
fa.fit(datos_escalados)

FactorAnalyzer(n_factors=1, rotation_kwargs={})

In [ ]:
pd.DataFrame(fa.loadings_)

,0
0,-0.822986
1,0.334364
2,-1.014525
3,-0.974734


In [ ]:
from scipy.sparse import data
#Se combierte el array a dataframe para añadir nombers a los ejes. 
#vector de loadings, cargas
pd.DataFrame(
    data    = fa.loadings_,
    columns = ['Factor1'],
    index   = x.columns
)

,Factor1
Sepal.Length,-0.822986
Sepal.Width,0.334364
Petal.Length,-1.014525
Petal.Width,-0.974734


En este resultado(el de arriba) se 
puede observar que se construyó únicamente un factor con la función 
FactorAnalyzer(n_factor = 1), y extrajimos las cargas para ver que información recolecta este 
nuevo factor, podemos ver que los valores con respecto a las cargas que están más 
cercanos son la longitud y el ancho de los pétalos, por lo tanto, podríamos decir  que el 
factor latente sería estructura de los pétalos.   

In [ ]:
#El siguiente escenario corresponde a construir 2 factores, veamos qué nos dicen las cargas:  
#Segundo Escenario
fa2 = FactorAnalyzer(n_factors = 2)
fa2.fit(datos_escalados)

FactorAnalyzer(n_factors=2, rotation_kwargs={})

In [ ]:
pd.DataFrame(fa2.loadings_)

,0,1
0,0.941751,0.173090
1,0.012016,1.001381
2,0.955574,-0.131196
3,0.919831,-0.085015


In [ ]:
pd.DataFrame(
    data    = fa2.loadings_,
    columns = ['Factor1', 'Factor2'],
    index   = x.columns
)

,Factor1,Factor2
Sepal.Length,0.941751,0.173090
Sepal.Width,0.012016,1.001381
Petal.Length,0.955574,-0.131196
Petal.Width,0.919831,-0.085015


(lo de arriba)
El primer factor ya que incluye información de los 
pétalos y de la longitud del sépalo, sin embargo, el segundo factor abarca información 
relacionada con el ancho del sépalo, por lo tanto, esto nos indica que el ancho del sépalo 
es un factor muy importante para el análisis, tanto que pudiéramos manejarlo de manera 
independiente de las demás variables y las demás considerarlas como un factor latente.   

In [ ]:
# Primer escenario
fa.get_factor_variance()

(array([2.76847435]), array([0.69211859]), array([0.69211859]))

In [ ]:
# Segundo escenario
fa2.get_factor_variance()

(array([2.64625105, 1.05716444]),
 array([0.66156276, 0.26429111]),
 array([0.66156276, 0.92585387]))

Con estos resultados, podemos observar la varianza explicada acumulada en los dos 
escenarios; el primer escenario nos muestra que el nuevo factor explica el 69.21% de la 
varianza, un valor un poco bajo, sin embargo, el segundo escenario muestra una varianza 
acumulada del 92.58%, por lo tanto, el mejor escenario sería el número 2.   

In [ ]:
datos_fa= pd.DataFrame(fa2.transform(datos_escalados))
datos_fa

,0,1
0,-1.280154,1.027878
1,-1.499788,-0.122744
2,-1.432940,0.341125
3,-1.270571,0.123532
4,-1.199130,1.262467
...,...,...
145,0.534845,-0.199106
146,0.388466,-1.311932
147,0.745833,-0.161202
148,1.075833,0.753056


In [ ]:
conjunto_fa = pd.concat([datos_fa, datos[['Species']]], axis = 1)
conjunto_fa

,0,1,Species
0,-1.280154,1.027878,setosa
1,-1.499788,-0.122744,setosa
2,-1.432940,0.341125,setosa
3,-1.270571,0.123532,setosa
4,-1.199130,1.262467,setosa
...,...,...,...
145,0.534845,-0.199106,virginica
146,0.388466,-1.311932,virginica
147,0.745833,-0.161202,virginica
148,1.075833,0.753056,virginica


In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
conjunto_fa.to_csv('conjunto_fa.csv')
!cp conjunto_fa.csv "/content/drive/MyDrive/Colab Notebooks/Modulo 6"